In [366]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Decorators and closures

## Closure
> A closure is a function with an extended scope that encompasses nonglobal variables referenced in the body of the function but not defined there

In [314]:
# implementing LCG with closure

def make_lcg(m, a, c, seed):
    x = seed
    
    def inner():
        nonlocal x
        x = (a * x + c) % m
        return x
    
    return inner


lcg = make_lcg(m=2 ** 32, a=1103515245, c=12345, seed=888)
[lcg() for _ in range(10)]

[669006417,
 4158894774,
 1529527223,
 2095963940,
 1696410253,
 2792375362,
 2678414419,
 3602631056,
 448717449,
 2336383630]

In [315]:
lcg.__code__.co_varnames

()

In [316]:
lcg.__code__.co_freevars

('a', 'c', 'm', 'x')

In [317]:
lcg.__closure__

(<cell at 0x7f5406ab9f50: int object at 0x7f5406af2d70>,
 <cell at 0x7f5406ab9950: int object at 0x7f5406ad7030>,
 <cell at 0x7f5406ab94d0: int object at 0x7f5406ad7250>,
 <cell at 0x7f5406ab91d0: int object at 0x7f5406ad7230>)

In [318]:
[cell.cell_contents for cell in lcg.__closure__]

[1103515245, 12345, 4294967296, 2336383630]

## Scopes
*LEGB Rule*
1. **Local (or function) scope** is the code block or body of any Python function or lambda expression
1. **Enclosing (or nonlocal) scope** is a special scope that only exists for nested functions.
1. **Global (or module) scope** is the top-most scope in a Python program, script, or module
1. **Built-in scope** is a special Python scope that’s created or loaded whenever you run a script or open an interactive session

In [327]:
# local
def cube(base):
    result = base ** 3
    print(f'The cube of {base} is: {result}')

In [328]:
cube.__code__.co_varnames

('base', 'result')

In [329]:
cube.__code__.co_argcount

1

In [330]:
cube.__code__.co_consts

(None, 3, 'The cube of ', ' is: ')

In [366]:
cube.__code__.co_name

'cube'

In [367]:
cube.__code__.co_freevars

()

In [499]:
# global
dir()

In [498]:
# built-in
dir(__builtins__)

## Decorator
A callable that takes another callable as argument

In [339]:
def logged(func):

    def inner(x):
        print(f"Running function {func.__name__} with arguments x={x}")
        return func(x)

    return inner

In [340]:
def bar(x):
    """Returns x"""
    return x

bar = logged(bar)
bar(42)

Running function bar with arguments x=42


42

In [342]:
@logged
def bar(x):
    return x

bar(42)

Running function bar with arguments x=42


42

In [343]:
bar.__name__

'inner'

In [340]:
bar.__doc__

Metadata are lost!

In [345]:
import functools

def logged(func):

    @functools.wraps(func)
    def inner(x):
        print(f"Running function {func.__name__} with arguments x={x}")
        func(x)

    return inner


@logged
def bar(x):
    """Returns x"""
    return x


bar(42)

Running function bar with arguments x=42


In [346]:
bar.__doc__

'Returns x'

In [347]:
bar.__name__

'bar'

## Stacked decorators

In [348]:
def dec1(f):
    
    def inner(x):
        print(f'Running {f.__name__} with dec1')
        return f(x)
    
    return inner


def dec2(f):
    
    def inner(x):
        print(f'Running {f.__name__} with dec2')
        return f(x)
    
    return inner


@dec1
@dec2
def bar(x):
    print('calling bar')

bar(42)


def bar(x):
    print('calling bar')

dec1(dec2(bar))(x)

Running inner with dec1
Running bar with dec2
calling bar
Running inner with dec1
Running bar with dec2
calling bar


## Parameterized decorator

What if user wants to pass in the file handle where the logs will be outputed to, such as stderr?

In [375]:
import sys
import functools


def logged(file=sys.stdout):
    
    def decorate(func):

        @functools.wraps(func)
        def inner(x):
            print(f'Running {func.__name__} with dec1', file=file)
            return func(x)
    
        return inner
    
    return decorate


@logged(file=sys.stderr)
def bar(x):
    return x


bar(42)

Running bar with dec1


42

In [374]:
# equivalently
class Logged:
    
    def __init__(self, file=sys.stdout):
        self.file = file
        
    def __call__(self, func):

        @functools.wraps(func)
        def inner(x):
            print(f'Running {func.__name__} with dec1', file=self.file)
            return func(x)
    
        return inner


@Logged(file=sys.stderr)
def bar(x):
    return x


bar(42)

Running bar with dec1


42

# Iterable, iterators and generators

## Iterable and Iterator

**Python obtains iterators from iterables.**

In [221]:
s = 'ABC'
for char in s:
    print(s)

ABC
ABC
ABC


In [222]:
s = 'ABC'

it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

A
B
C


Interface of an iterator: 
    
`__next__`: returns the next available item, and raise StopIteration when there are no more. 

`__iter__`: returns self. To be used in e.g. for loop

Interface of an iterable:

`__iter__`: returns an iterator. If not implemented, `__getitem__` will be used (for backward compatibility).

In [236]:
# implementing LCG with iterator

class LCG:
    
    def __init__(self, m, a, c, seed):
        self.m = m
        self.a = a
        self.c = c
        self.x = seed
        
    def __iter__(self):
        return self
    
    def __next__(self):
        self.x = (self.a * self.x + self.c) % self.m
        return self.x

    
    
lcg = LCG(m=2 ** 32, a=1103515245, c=12345, seed=888)

for i, x in enumerate(lcg):
    if i > 5:
        break
    print(x)

669006417
4158894774
1529527223
2095963940
1696410253
2792375362


## Generator

In [251]:
def gen_123():
    yield 1
    yield 2
    yield 3

In [252]:
for i in gen_123():
    print(i)

1
2
3


In [253]:
g = gen_123()
print(type(gen_123))
print(type(g))

print(next(g))
print(next(g))
print(next(g))
print(next(g))

<class 'function'>
<class 'generator'>
1
2
3


StopIteration: 

In [256]:
def gen_123():
    print('Returning 1')
    yield 1
    print('Returning 2')
    yield 2
    print('Returning 3')
    yield 3
    print('Finishing up')

In [257]:
g = gen_123()

In [258]:
next(g)

Returning 1


1

In [261]:
next(g)

Returning 2


2

In [262]:
next(g)

Returning 3


3

In [263]:
next(g)

Finishing up


StopIteration: 

Implementing LCG with generator

In [238]:
def lcg(m, a, c, seed):
    x = seed
    while True:
        x = (a * x + c) % m
        yield x


for i, x in enumerate(lcg(m=2 ** 32, a=1103515245, c=12345, seed=888)):
    if i > 5:
        break
    print(x)

669006417
4158894774
1529527223
2095963940
1696410253
2792375362


## Generators in built-in functions

In [266]:
z = zip([1, 2, 3], ['a', 'b', 'c'])
print(z)

In [268]:
reversed([1, 2, 3])

In [269]:
filter(lambda x: x > 5, range(10))